In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data = pd.read_csv("../input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("../input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)
print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [6]:
# features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Fare"]
# sns.pairplot(data=train_data[features])

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [8]:
def get_value(best, X_train, y_train, X_tes, y_tes):
    model = RandomForestClassifier(n_estimators=best, random_state=1)
    model.fit(X_train, y_train)
    mea = mean_absolute_error(model.predict(X_tes), y_tes)
    return mea
train_data.Fare.fillna(train_data['Fare'].mean(), inplace=True)
train_data.Age.fillna(train_data['Age'].mean(), inplace=True)
train_data.Age = train_data.Age.round()
test_data.Fare.fillna(test_data['Fare'].mean(), inplace=True)
test_data.Age.fillna(test_data['Age'].mean(), inplace=True)
test_data.Age = test_data.Age.round()
#train_data.Age = train_data.Age.astype(int)
#train_data.isnull().value_counts()
train_data.Age.sort_values
#sns.distplot(a=train_data.Age)

<bound method Series.sort_values of 0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888    30.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64>

In [9]:
y = train_data["Survived"]
temp = pd.get_dummies(train_data['Sex'])
tempo = pd.get_dummies(test_data['Sex'])
features = ["Pclass", "SibSp", "Parch", "Age","Fare", "male", "female"]
train_data['male'] = temp['male']
train_data[ 'female'] = temp['female'] 
train_data
X = train_data[features]
test_data['male'] = tempo['male']
test_data[ 'female'] = tempo['female'] 
train_data
X_test = test_data[features]
X_train, val_X, y_train, val_y = train_test_split(X, y, test_size= 0.3, random_state=1)
X_test.isnull().value_counts()

Pclass  SibSp  Parch  Age    Fare   male   female
False   False  False  False  False  False  False     418
dtype: int64

In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
val_X = sc.transform(val_X)
X_test = sc.transform(X_test)
X = sc.transform(X)

In [11]:
# df = pd.DataFrame(columns=['MEA', 'n-value'])
# for i in range(1,10):
#     my_mea = get_value(i,X_train, y_train, val_X, val_y)
#     df.loc[i, 'MEA'] = my_mea
#     df.loc[i, 'n-value'] = i
# df

In [12]:
# df.plot(x='n-value', y='MEA', figsize=(22,15))

In [13]:
model = RandomForestClassifier(n_estimators=6, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

In [14]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
